In [ ]:
from dotenv import load_dotenv, find_dotenv
from langchain_openai import OpenAI
import os


## Get environ variables

In [ ]:
load_dotenv(find_dotenv(), override=True)

#os.environ.get('OPENAI_API_KEY')

## Chat Models with GPT3.5 and GPT4

In [ ]:
#from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI() #this will use default gpt-3.5-turbo and temperature 0.7
#llm = ChatOpenAI(model='gpt-4-turbo-preview')

output = llm.invoke('Explain quantum computing in one sentence.')
print(output.content)

In [ ]:
help(ChatOpenAI)

In [ ]:
from langchain.schema import(
    SystemMessage,
    AIMessage,
    HumanMessage
)

messages = [
    SystemMessage(content="You are a physicist.  Respond only in german."),
    HumanMessage(content="Explain quantum computing in one sentence.")
]

output = llm.invoke(messages)
print(output.content)

## Caching LLM Responses

### In Memory-Cache

In [ ]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI

llm = OpenAI(model='gpt-3.5-turbo-instruct')

In [ ]:
%%time

from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())

prompt = 'Tell me a joke that a toddler can understand.'
llm.invoke(prompt)

In [ ]:
%%time
llm.invoke(prompt)

### SQLlite Cachain

In [ ]:
%%time
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path='resources/cache.db'))

# First request (not in cache, will take longer)
llm.invoke('tell me a joke in italian')

# Second request  (cached, faster)
llm.invoke('tell me a joke in italian')

## Streaming

In [ ]:
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI()


prompt = 'write a rock song about moon and a raven'

print(llm.invoke(prompt).content)


In [ ]:
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)


# Prompt Templates

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = '''You are a experienced virologist.  Write a few sentences about the following virus {virus} in {language}.
'''

prompt_template = PromptTemplate.from_template(template=template)

prompt = prompt_template.format(virus='SARS-CoV-2', language='french')

prompt

In [ ]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)

print(output.content)

## ChatPromptTemplates

In [ ]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage


chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in json format'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population')
    ]
)

messages = chat_template.format_messages(n=5, area='World')

print(messages)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

output = llm.invoke(messages)
print(output.content)


# Simple Chains

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain


llm = ChatOpenAI()

template = '''You are a experienced virologist.  Write a few sentences about the following virus {virus} in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm, 
    prompt=prompt_template,
    verbose=True
)

output = chain.invoke({'virus': 'SARS-CoV-2', 'language': 'french'})


In [ ]:
print(output)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain


llm = ChatOpenAI()

template = 'What is the capital of {country}? List the top 3 places to visit in that city. Use bullet points.'
prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm, 
    prompt=prompt_template,
    verbose=True
)

country = input('Enter a country: ')
output = chain.invoke({'country': country})
print(output['text'])


## Sequential Chains

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.5)
prompt_template1 = PromptTemplate.from_template(
    template = 'You are an experienced scientist and Python programmer.  Write a function that implements the concept of {concept}.'
)

chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

llm2 = ChatOpenAI(model='gpt-4-turbo-preview', temperature=1.2)

prompt_template2 = PromptTemplate.from_template(
    template = 'Given the the Python function {function}, describe it as detailed as possible'
)

chain2 = LLMChain(llm=llm2, prompt=prompt_template2)    


overall_chain = SimpleSequentialChain(chains = [chain1, chain2], verbose=True)

output = overall_chain.invoke('linear regression')

In [ ]:
print(output['output'])

## Langchain Agents in Action: Python REPL
### Read-Eval-Print Loop

In [ ]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()

python_repl.run('print([n for n in range (1,100) if n % 13 == 0])')

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

agent_executor = create_python_agent(
    llm=llm, 
    tool=PythonREPLTool(), 
    verbose=True
)

agent_executor.invoke('Calculate the square root of the factorial 12 and display it with 4 decimal points')


In [ ]:
response = agent_executor.invoke('What is the answer to 5.1 ** 7.3')

In [ ]:
response

In [ ]:
# this the value of response:
# {'input': 'What is the answer to 5.1 ** 7.3', 'output': '146306.05007233328'}

print(response['input'])

print(response['output'])

## Langchain Tool: DuckDuckGo and Wikipedia

In [ ]:
from langchain.tools import DuckDuckGoSearchRun 

search = DuckDuckGoSearchRun()

output = search.invoke('Where was Freddie Mercury born?')

print(output)


In [ ]:
search.name

In [ ]:
search.description

In [ ]:
from langchain.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()

output = search.run('Freddie Mercury and Queen')

print(output)

In [ ]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(region = 'de-de', max_results = 3, safesearch = 'moderate')
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source = 'news')

output=search.run('Berlin')

print(output)


## Creating a ReAct Agent

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv()

True

In [4]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import AgentExecutor, Tool, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI
from langchain.utilities import WikipediaAPIWrapper

In [5]:
llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

template = ''''
Answer the following questions as best you can.  
Questions: {q}
'''

prompt_template=PromptTemplate.from_template(template)
prompt=hub.pull('hwchase17/react')
print(type(prompt))

print(prompt.input_variables)

<class 'langchain_core.prompts.prompt.PromptTemplate'>
